## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the loss between targets and totals of aggregating the targeted variables across all data records.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.
- `estimate_matrix` (pd.DataFrame): matrix representing the contribution of each record to a given variable total.
- `estimate_function` (Callable): function that produces the estimate values for each targeted variable based on the weights and the contribution of each record to said targeted variable. The standard way of doing it if not provided is `estimate_matrix @ weights`.

Calibration can be easily done by initializing the `Calibration` class, and passing in the parameters above. Then the `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- Updates `self.weights` with the calibrated results 
- Produces a calibration log with performance metrics

This module also supports regularization in case a sparse matrix that optimizes to reduce the data size simultaneously to calibration is desired. To use this functionality pass `regularize=True` to the `calibrate()` call. The method will update `self.sparse_weights` with the sparse calibrated results, which can then be used to drop records with a weight close to 0.

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [5]:
from microcalibrate.calibration import Calibration
import logging
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

calibration_logger = logging.getLogger("microcalibrate.calibration")
calibration_logger.setLevel(logging.WARNING)

# Create a sample dataset with age and income data
random_generator = np.random.default_rng(0)
data = pd.DataFrame({
    "age": np.append(random_generator.integers(18, 70, size=120), 71), 
    "income": random_generator.normal(40000, 10000, size=121),
})

# Set initial weights (all one in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
    "income_aged_71" : (data["age"] == 71).astype(float) * data["income"],
})

# 15% higher than the sum of data with the original weights
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1000).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_71"] * weights * 1.15).sum()
])

print(f"Original weights: {weights}")
print(f"Original targets: {targets}")

Original weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Original targets: [7.37032429e+08 9.76779350e+05 4.36479914e+04]


In [6]:
# Initialize the Calibration object
calibrator = Calibration(
    estimate_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=528,
    learning_rate=0.01,
    dropout_rate=0,
)

# Perform the calibration
performance_df = calibrator.calibrate()

print(f"Original dataset size: {len(targets_matrix)}")
print(f"Calibrated dataset size: {len(calibrator.estimate_matrix)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

Target income_aged_20_30 (7.37e+08) differs from initial estimate (7.37e+05) by 3.00 orders of magnitude.
Target income_aged_71 is supported by only 0.83% of records in the loss matrix. This may make calibration unstable or ineffective.
Reweighting progress: 100%|██████████| 528/528 [00:00<00:00, 2153.20epoch/s, loss=1.61e-11, weights_mean=150, weights_std=357, weights_min=1]

Original dataset size: 121
Calibrated dataset size: 121
Number of calibrated weights: 121


## Pre-calibration target assessment

Before running the calibration, it's important to understand whether your targets are achievable and well-posed. The Calibration class provides a key method for this:

**`assess_analytical_solution()`** - Analyzes the mathematical difficulty of achieving your target combination

Additionally, you should manually check for:
- Order of magnitude differences between targets
- Highly correlated targets
- Redundant or conflicting constraints

### Analytical solution assessment

The analytical solution assessment examines the optimization difficulty by using the Moore-Penrose inverse for a least squares solution. It shows:
- How the loss increases as each target is added
- Which targets contribute most to the optimization difficulty
- Targets with large delta_loss values that complicate calibration

This is particularly useful when you have many correlated targets or when targets overlap (e.g., total income includes all regional incomes).

In [7]:
# Assess the analytical solution before calibration
print("Assessing analytical solution feasibility...")
analytical_assessment = calibrator.assess_analytical_solution()

# Display the assessment results
print("\n" + "="*60)
print("ANALYTICAL SOLUTION ASSESSMENT")
print("="*60)
print("\nThis shows how the loss increases as each target is added:")
print(analytical_assessment.to_string(index=False))
print("\nTargets with large delta_loss values complicate the optimization problem.")

Assessing analytical solution feasibility...

ANALYTICAL SOLUTION ASSESSMENT

This shows how the loss increases as each target is added:
     target_added         loss    delta_loss
income_aged_20_30 0.000000e+00           NaN
income_aged_40_50 6.776264e-21  6.776264e-21
   income_aged_71 4.535156e-21 -2.241108e-21

Targets with large delta_loss values complicate the optimization problem.


### Understanding the assessment

The assessment provides several key insights:

1. **Condition number**: Values above 1e10 suggest numerical instability. High condition numbers mean small changes in targets can lead to large changes in weights.

2. **Rank analysis**: If the rank is less than the number of targets, some targets are redundant or conflicting.

3. **Recommendations**: The assessment provides specific guidance based on the analysis.

### Target tolerance checking

Another important pre-calibration check is to ensure your targets are on similar scales. Targets that differ by many orders of magnitude can cause convergence issues.

In [8]:
# Check for order of magnitude differences in targets
print("Checking target scales...")

# Manually check for large differences in target scales
target_values = targets
target_names = ["income_aged_20_30", "income_aged_40_50", "income_aged_71"]

# Calculate order of magnitude for each target
orders_of_magnitude = np.log10(np.abs(target_values) + 1e-10)
median_order = np.median(orders_of_magnitude)

print("\nTarget scale analysis:")
for i, name in enumerate(target_names):
    order_diff = orders_of_magnitude[i] - median_order
    print(f"  {name}: {target_values[i]:.2e} (order difference from median: {order_diff:.1f})")
    
# Identify targets with extreme scale differences
large_scale_diff = np.abs(orders_of_magnitude - median_order) > 3
if np.any(large_scale_diff):
    print("\n⚠️ WARNING: The following targets differ by more than 3 orders of magnitude from the median:")
    for i in np.where(large_scale_diff)[0]:
        print(f"  - {target_names[i]}: {target_values[i]:.2e}")
    print("\nConsider rescaling these targets for better numerical stability.")
else:
    print("\n✓ All targets are within reasonable scale differences")

Checking target scales...

Target scale analysis:
  income_aged_20_30: 7.37e+08 (order difference from median: 2.9)
  income_aged_40_50: 9.77e+05 (order difference from median: 0.0)
  income_aged_71: 4.36e+04 (order difference from median: -1.3)

✓ All targets are within reasonable scale differences


### Interpreting tolerance warnings

The tolerance check identifies targets that:
- Differ by more than 3 orders of magnitude from the median target
- May cause numerical instability during optimization
- Should potentially be rescaled or reviewed

In our example, we have one target that is much larger than the others (income_aged_20_30), which the system warns about. This is expected since we artificially multiplied it by 1000 in our setup.

### Excluded targets

If the pre-calibration assessment identifies problematic targets, you can exclude them from calibration:

In [16]:
# Example: Excluding problematic targets
# If you identify targets that cause issues, you can exclude them:

# Create a calibrator with excluded targets
calibrator_with_exclusions = Calibration(
    estimate_matrix=targets_matrix,
    weights=weights.copy(), 
    targets=targets,
    excluded_targets=["income_aged_71"],  # Exclude the smallest target
    noise_level=0.05,
    epochs=100,
    learning_rate=0.01,
)

print("Calibration setup with excluded targets:")
print(f"Original targets: {calibrator_with_exclusions.original_target_names}")
print(f"Excluded targets: {calibrator_with_exclusions.excluded_targets}")
print(f"Active targets for calibration: {calibrator_with_exclusions.target_names}")
print(f"Number of active targets: {len(calibrator_with_exclusions.targets)}")

Calibration setup with excluded targets:
Original targets: ['income_aged_20_30' 'income_aged_40_50' 'income_aged_71']
Excluded targets: ['income_aged_71']
Active targets for calibration: ['income_aged_20_30' 'income_aged_40_50']
Number of active targets: 2


## Best practices for pre-calibration assessment

1. **Always run analytical assessment first** - This helps identify fundamental issues with your target specification before spending time on calibration.

2. **Check target scales** - Targets that differ by many orders of magnitude should be rescaled or normalized to improve convergence.

3. **Look for redundant targets** - If targets are highly correlated or mathematically dependent, consider removing redundant ones.

4. **Consider the degrees of freedom** - Having more targets than observations makes exact calibration impossible. The system will find a best-fit solution.

5. **Use exclusions strategically** - Temporarily exclude problematic targets to get initial calibration working, then gradually add them back.

6. **Monitor condition numbers** - High condition numbers (>1e10) indicate numerical instability. Consider reformulating your targets or adding regularization.

Now let's proceed with the actual calibration:

In [11]:
# Calculate final weighted totals
final_totals = targets_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [7.37032429e+08 9.76779350e+05 4.36479914e+04]
Final calibrated totals: [7.37025243e+08 9.76778358e+05 4.36469951e+04]
Difference: [-7.18606246e+03 -9.91859882e-01 -9.96308503e-01]
Relative error: [-0.000975   -0.00010154 -0.0022826 ]


In [12]:
np.testing.assert_allclose(
        final_totals,
        targets,
        rtol=0.01,  # relative tolerance
        err_msg="Calibrated totals do not match target values",
    )

In [13]:
performance_df.head()

,epoch,loss,target_name,target,estimate,error,abs_error,rel_abs_error
0,0,0.339299,income_aged_20_30,7.370324e+08,7.595331e+05,-7.362729e+08,7.362729e+08,0.998969
1,0,0.339299,income_aged_40_50,9.767794e+05,8.723239e+05,-1.044555e+05,1.044555e+05,0.106939
2,0,0.339299,income_aged_71,4.364799e+04,3.961747e+04,-4.030520e+03,4.030520e+03,0.092341
3,52,0.332147,income_aged_20_30,7.370324e+08,1.320273e+06,-7.357122e+08,7.357122e+08,0.998209
4,52,0.332147,income_aged_40_50,9.767794e+05,9.774676e+05,6.882500e+02,6.882500e+02,0.000705


In [14]:
g20 = performance_df.query("target_name == 'income_aged_20_30'")
g40 = performance_df.query("target_name == 'income_aged_40_50'")

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Estimate vs target: income_aged_20_30",
        "Estimate vs target: income_aged_40_50",
        "Relative absolute error: income_aged_20_30",
        "Relative absolute error: income_aged_40_50",
    ],
    shared_xaxes=True,
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["target"],
        mode="lines", name="Target 20-30",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["estimate"],
        mode="lines", name="Estimate 20-30",
        line=dict(color="blue"),
    ),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["target"],
        mode="lines", name="Target 40-50",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=2,
)
fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["estimate"],
        mode="lines", name="Estimate 40-50",
        line=dict(color="green"),
    ),
    row=1, col=2,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="blue"),
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="green"),
    ),
    row=2, col=2,
)

fig.update_layout(
    height=800, width=1050,
    title_text="Calibration performance over epochs",
    legend=dict(x=1.05, y=1, xanchor="left", yanchor="top"),
    margin=dict(r=200),
)
fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.update_xaxes(title_text="Epoch", row=2, col=2)
fig.update_yaxes(title_text="Income ($)", row=1, col=1)
fig.update_yaxes(title_text="Income ($)", row=1, col=2)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=1)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=2)

fig.show()

In [15]:
summary = calibrator.summary()
summary

,Metric,Official target,Final estimate,Relative error
0,income_aged_20_30,7.370324e+08,7.370273e+08,-0.000007
1,income_aged_40_50,9.767794e+05,9.767784e+05,-0.000001
2,income_aged_71,4.364799e+04,4.364699e+04,-0.000023


## Summary

The Calibration class provides comprehensive tools for survey weight calibration:

1. **Pre-calibration assessment**:
   - Analytical solution feasibility analysis
   - Target tolerance checking
   - Correlation analysis
   - Target exclusion capabilities

2. **Standard calibration**:
   - Gradient-based optimization
   - Multi-target support
   - Progress monitoring
   - Performance logging

3. **Advanced features**:
   - L0 regularization for sparse weights
   - Hyperparameter tuning (see [hyperparameter tuning notebook](hyperparameter_tuning.ipynb))
   - Robustness evaluation (see [robustness evaluation notebook](robustness_evaluation.ipynb))

By using the pre-calibration assessment tools, you can identify and address potential issues before running the calibration, saving time and improving results. The L0 regularization feature allows you to maintain calibration accuracy while significantly reducing dataset size, which is valuable for large-scale applications.